<a href="https://colab.research.google.com/github/shashvindu/Pyspark/blob/main/pysaprksetup_testwithretaildata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#sc.stop()

In [7]:
'''!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark  '''


'!apt-get install openjdk-8-jdk-headless -qq > /dev/null\n!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz\n!tar xf spark-3.3.0-bin-hadoop3.tgz\n!pip install -q findspark\n!pip install pyspark  '

In [8]:
#!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

In [9]:
from pyspark import SparkContext

In [10]:
sc=SparkContext()

In [11]:
a=sc.parallelize([1,2,3])

In [12]:
a.take(2)

[1, 2]

In [13]:
sc.stop()

In [14]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

retails_data_csv = "/content/drive/MyDrive/data/retail-data/by-day/*.csv"

In [15]:
spark = SparkSession \
    .builder \
    .appName("Basic Dataframe Operations") \
    .config("spark.master", "local") \
    .getOrCreate()

In [16]:
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load(retails_data_csv)\
  .repartition(5)

In [17]:
df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580729|    22492|MINI PAINT SET VI...|       1|2011-12-05 17:24:00|     1.63|      null|United Kingdom|
|   575070|    21756|BATH BUILDING BLO...|       1|2011-11-08 12:58:00|     6.25|   17841.0|United Kingdom|
|   574573|    22945|CHRISTMAS METAL T...|       6|2011-11-04 16:35:00|     0.85|   14592.0|United Kingdom|
|   575911|    35970|ZINC FOLKART SLEI...|      24|2011-11-11 16:13:00|     1.69|   17314.0|United Kingdom|
|   576559|    22360|GLASS JAR ENGLISH...|       1|2011-11-15 13:19:00|     2.95|   14711.0|United Kingdom|
|   569900|    35970|ZINC FOLKART SLEI...|      12|2011-10-06 16:17:00|     1.69|   17651.0|United Kingdom|
|   539453|    21365|MIRRORE

In [18]:
df.rdd.getNumPartitions()

5

In [19]:
#sql table
df.createOrReplaceTempView("dfTable")

In [20]:
from pyspark.sql.functions import count, countDistinct, approx_count_distinct
df.select(count("StockCode")).show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



In [21]:
df.groupBy("StockCode").count().show()

+---------+-----+
|StockCode|count|
+---------+-----+
|    22596|  274|
|    21249|  119|
|    21894|  135|
|    22121|  141|
|    23318|  391|
|    20868|   46|
|    22728|  810|
|    23459|   21|
|    90022|   21|
|    22254|   61|
|    21259|  296|
|    21452|  200|
|   90210B|    7|
|    90143|   22|
|   90197B|   27|
|    21248|   68|
|    21889|  607|
|   85132b|    9|
|    21331|    8|
|   85115B|    1|
+---------+-----+
only showing top 20 rows



In [22]:
df.select(countDistinct("StockCode")).show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [23]:
df.select(approx_count_distinct("StockCode", 0.1)).show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



In [24]:
spark.sql("""SELECT count(StockCode),
                    approx_count_distinct(StockCode)
             FROM dfTable""").show()

+----------------+--------------------------------+
|count(StockCode)|approx_count_distinct(StockCode)|
+----------------+--------------------------------+
|          541909|                            3804|
+----------------+--------------------------------+



In [25]:
sqlc="SELECT count(StockCode), approx_count_distinct(StockCode)  FROM dfTable"
spark.sql(sqlc).show()

+----------------+--------------------------------+
|count(StockCode)|approx_count_distinct(StockCode)|
+----------------+--------------------------------+
|          541909|                            3804|
+----------------+--------------------------------+



In [26]:
#  Example 2 - first, last, min, max
from pyspark.sql.functions import first, last, min, max

In [27]:
df.select(first("StockCode"), last("StockCode")).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|           22148|          21201|
+----------------+---------------+



In [28]:
df.select(min("Quantity"), max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [29]:
spark.sql("""SELECT first(StockCode) as first, 
                    last(StockCode) as last,
                    min(Quantity)  as minQty,
                    max(Quantity) as maxQty
             FROM dfTable""").show()

+-----+-----+------+------+
|first| last|minQty|maxQty|
+-----+-----+------+------+
|22578|22641|-80995| 80995|
+-----+-----+------+------+



In [30]:
#  Example 3 - sum, sumDistinct, avg
from pyspark.sql.functions import sum, sumDistinct, avg

In [31]:
df.select(sum("Quantity")).show() 

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



In [32]:
df.select(sumDistinct("Quantity")).show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/functions.py:315: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)


+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



In [33]:
spark.sql("""SELECT sum(Quantity)  as sumQty, 
                    mean(Quantity) as mean
             FROM dfTable""").show() 

+-------+----------------+
| sumQty|            mean|
+-------+----------------+
|5176450|9.55224954743324|
+-------+----------------+



In [34]:
from pyspark.sql.functions import mean, expr 

In [35]:
df.select(
  count("Quantity").alias("total_transactions"),
  sum("Quantity").alias("total_purchases"),
  avg("Quantity").alias("avg_purchases"),
  expr("mean(Quantity)").alias("mean_purchases")) \
.selectExpr(
  "total_purchases/total_transactions",
  "avg_purchases",
  "mean_purchases").show()

+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



In [36]:
# Example 4 - varience and standard deviation
from pyspark.sql.functions import var_pop, stddev_pop, variance, stddev
from pyspark.sql.functions import var_samp, stddev_samp

In [37]:
df.select(variance("Quantity"), stddev("Quantity"),
  var_pop("Quantity"), var_samp("Quantity"),
  stddev_pop("Quantity"), stddev_samp("Quantity")).show()

+------------------+---------------------+-----------------+------------------+--------------------+---------------------+
|var_samp(Quantity)|stddev_samp(Quantity)|var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+---------------------+-----------------+------------------+--------------------+---------------------+
| 47559.39140929852|   218.08115785023364|47559.30364660883| 47559.39140929852|  218.08095663447745|   218.08115785023364|
+------------------+---------------------+-----------------+------------------+--------------------+---------------------+



In [38]:
spark.sql("""SELECT var_pop(Quantity), 
                    var_samp(Quantity),
                    stddev_pop(Quantity), 
                    stddev_samp(Quantity)
             FROM dfTable""").show()

+-----------------+------------------+--------------------+---------------------+
|var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+-----------------+------------------+--------------------+---------------------+
|47559.30364660883| 47559.39140929852|  218.08095663447745|   218.08115785023364|
+-----------------+------------------+--------------------+---------------------+



In [39]:
# Example 5 - skewness & kurtosis
from pyspark.sql.functions import skewness, kurtosis

In [40]:
df.select(skewness("Quantity"), kurtosis("Quantity")).show()  

+--------------------+------------------+
|  skewness(Quantity)|kurtosis(Quantity)|
+--------------------+------------------+
|-0.26407557610430366|119768.05495533711|
+--------------------+------------------+



In [41]:
spark.sql("SELECT skewness(Quantity), kurtosis(Quantity) FROM dfTable").show()

+--------------------+------------------+
|  skewness(Quantity)|kurtosis(Quantity)|
+--------------------+------------------+
|-0.26407557610430366|119768.05495533711|
+--------------------+------------------+



In [42]:
# Example 6 - Covariance and Correlation
from pyspark.sql.functions import corr, covar_pop, covar_samp

In [43]:
df.select(corr("InvoiceNo", "Quantity"), covar_samp("InvoiceNo", "Quantity"),
    covar_pop("InvoiceNo", "Quantity")).show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085633174E-4|             1052.7280543896418|            1052.7260778735376|
+-------------------------+-------------------------------+------------------------------+



In [44]:
spark.sql("""SELECT corr(InvoiceNo, Quantity), 
                        covar_samp(InvoiceNo, Quantity),
                        covar_pop(InvoiceNo, Quantity)
                  FROM dfTable""").show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085633174E-4|             1052.7280543896418|            1052.7260778735376|
+-------------------------+-------------------------------+------------------------------+



In [45]:
#- Example 7 - Aggregation on Collections
from pyspark.sql.functions import collect_set, collect_list

In [46]:

df.agg(collect_set("Country"), collect_list("Country")).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [Switzerland, Swi...|
+--------------------+---------------------+



In [47]:
spark.sql("SELECT collect_set(Country), collect_list(Country) FROM dfTable") \
     .show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [Switzerland, Swi...|
+--------------------+---------------------+



In [48]:
#  Example 8 - Grouping
from pyspark.sql.functions import count, sum
df.groupBy("InvoiceNo", "CustomerId").count().show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   575753|   17841.0|   80|
|   580657|   14696.0|   20|
|   581474|   12748.0|   24|
|   570413|   17652.0|   18|
|   579062|   17651.0|   16|
|   576357|   15660.0|   18|
|  C579018|   16678.0|    1|
|   576641|   17549.0|    4|
|   580090|   13323.0|    5|
|   577338|   17650.0|    8|
|   574138|   12415.0|    5|
|   577783|   18139.0|   35|
|   581020|   17511.0|   40|
|   572703|   16110.0|  154|
|  C538084|   12649.0|    2|
|   568895|   15356.0|   54|
|  C577375|   14441.0|    2|
|   568670|   14978.0|   38|
|   580138|   13199.0|    3|
|   575220|   17511.0|   14|
+---------+----------+-----+
only showing top 20 rows



In [49]:
df.groupBy("InvoiceNo").agg(
    count("Quantity").alias("quan"),
    expr("count(Quantity)"))\
   .show()

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   575961|  13|             13|
|   575671|  20|             20|
|   577511|  46|             46|
|   576059|  44|             44|
|   550469|  31|             31|
|   577022|  38|             38|
|   576112|  20|             20|
|   569823|  69|             69|
|   578292|  72|             72|
|   575091|  38|             38|
|   577541|  21|             21|
|   536596|   6|              6|
|   574592|   8|              8|
|   578057|  28|             28|
|   578459|   8|              8|
|   580906|   4|              4|
|   567695|   1|              1|
|   537252|   1|              1|
|   570281|   3|              3|
|   570264|   1|              1|
+---------+----+---------------+
only showing top 20 rows



In [50]:
df.groupBy("InvoiceNo").agg(
    expr("avg(Quantity)"),expr("sum(Quantity)"),expr("stddev_pop(Quantity)"))\
  .show()

+---------+------------------+-------------+--------------------+
|InvoiceNo|     avg(Quantity)|sum(Quantity)|stddev_pop(Quantity)|
+---------+------------------+-------------+--------------------+
|   575961| 2.769230769230769|           36|  1.5268794800984007|
|   575671|             16.65|          333|  12.141972656862638|
|   577511|3.1739130434782608|          146|   5.402512892872719|
|   576059|2.8181818181818183|          124|   5.223516436936151|
|   550469| 5.129032258064516|          159|   8.838121007003883|
|   577022| 5.131578947368421|          195|  2.9034557688489158|
|   576112|              10.9|          218|   7.495998932764064|
|   569823|1.4782608695652173|          102|  0.9869980199409519|
|   578292| 5.902777777777778|          425|    8.75937548861888|
|   575091|11.552631578947368|          439|   5.008925551458656|
|   577541| 9.333333333333334|          196|    9.18245393767158|
|   536596|               1.5|            9|   1.118033988749895|
|   574592

In [51]:
from pyspark.sql.functions import col, to_date

In [52]:
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))

In [53]:
dfWithDate.createOrReplaceTempView("dfWithDate")
spark.sql("SELECT InvoiceDate, date FROM dfWithDate").show()

+-------------------+----------+
|        InvoiceDate|      date|
+-------------------+----------+
|2010-12-01 16:15:00|2010-12-01|
|2011-12-08 15:49:00|2011-12-08|
|2011-12-08 15:49:00|2011-12-08|
|2011-11-11 12:02:00|2011-11-11|
|2011-11-11 12:02:00|2011-11-11|
|2011-11-11 12:02:00|2011-11-11|
|2011-11-11 12:02:00|2011-11-11|
|2011-11-11 12:02:00|2011-11-11|
|2011-11-11 12:02:00|2011-11-11|
|2011-11-11 12:02:00|2011-11-11|
|2011-11-23 10:23:00|2011-11-23|
|2011-11-23 10:23:00|2011-11-23|
|2011-11-23 10:23:00|2011-11-23|
|2011-11-23 10:23:00|2011-11-23|
|2011-11-30 12:16:00|2011-11-30|
|2011-11-30 12:16:00|2011-11-30|
|2011-11-30 12:16:00|2011-11-30|
|2011-11-30 12:16:00|2011-11-30|
|2011-11-30 12:16:00|2011-11-30|
|2011-11-30 12:16:00|2011-11-30|
+-------------------+----------+
only showing top 20 rows



In [55]:
spark.stop()